In [ ]:
import pandas as pd
import mysql.connector
import pandas as pd
import mysql.connector

# Establish the connection to MySQL
mydb = mysql.connector.connect(
    host="127.0.0.1",  # Specify the host, commonly localhost
    user="root",       # Your MySQL username
    password="SHYAM@sk25"  # Your MySQL password
)

# Create a cursor object to execute SQL queries
cursor = mydb.cursor()

# Create the database 'REDBUS' if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS REDBUS")
print("Database 'REDBUS' created (or already exists)")

# Use the 'REDBUS' database
cursor.execute("USE REDBUS")

# Define the schema for the MySQL table
schema = """
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT PRIMARY KEY AUTO_INCREMENT,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    price DECIMAL(10,2),
    seats_available INT,
    star_rating FLOAT
);
"""
cursor.execute(schema)
print("Table 'bus_routes' created successfully!")

# List of CSV files to read
csv_files = [
    'himachal_bus_details.csv', 'south_bengal_bus_details.csv', 'uttar_pradesh_bus_details.csv',
    'andhra_pradesh_bus_details.csv', 'WEST_BENGAL_bus_details.csv', 'Telangana_bus_details.csv',
    'rajasthan_bus_details.csv', 'kerala_bus_details.csv', 'chandigarh_bus_details.csv', 'punjab_bus_details.csv'
]

# Combine data from all CSV files into a single DataFrame
dfs = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dfs, ignore_index=True)

# Rename columns to lowercase and match the database schema
combined_df.columns = combined_df.columns.str.lower()  # Normalize column names to lowercase

# Debug: Print column names to verify they match the schema
print("Columns in DataFrame after normalization:", combined_df.columns)

# Rename columns to match the database table schema
combined_df.rename(columns={
    "departing_time": "departing_time",
    "reaching_time": "reaching_time",
    "star_rating": "star_rating",
    "seat_availability": "seats_available",
    "route_name": "route_name",
    "route_link": "route_link",
    "bus_name": "busname",
    "bus_type": "bustype",
    "price": "price",
    "duration": "duration"
}, inplace=True)

# Clean and transform data

# Convert time columns to proper MySQL TIME format
for time_col in ['departing_time', 'reaching_time']:
    if time_col in combined_df.columns:
        combined_df[time_col] = pd.to_datetime(combined_df[time_col], format='%H:%M', errors='coerce').dt.time

# Clean the 'price' column
if 'price' in combined_df.columns:
    combined_df['price'] = (
        combined_df['price'].astype(str)
        .str.replace('INR ', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )

# Extract seat numbers from the 'seats_available' column
if 'seats_available' in combined_df.columns:
    combined_df['seats_available'] = (
        combined_df['seats_available']
        .astype(str)
        .str.extract(r'(\d+)')
        .fillna(0)
        .astype(int)
    )

# Drop rows with missing critical data
critical_columns = ['route_name', 'route_link', 'busname', 'bustype', 'departing_time', 'reaching_time', 'price', 'seats_available', 'star_rating']
combined_df = combined_df.dropna(subset=critical_columns)

# Debug: Check the cleaned DataFrame
print("Cleaned DataFrame preview:")
print(combined_df.head())

# Prepare data for insertion
columns_to_insert = [
    "route_name", "route_link", "busname", "bustype",
    "departing_time", "duration", "reaching_time",
    "price", "seats_available", "star_rating"
]
insert_query = f"""
    INSERT INTO bus_routes ({', '.join(columns_to_insert)}) 
    VALUES ({', '.join(['%s'] * len(columns_to_insert))})
"""
data = combined_df[columns_to_insert].values.tolist()

# Insert data into the 'bus_routes' table
cursor.executemany(insert_query, data)
mydb.commit()
print(f"Data inserted into table 'bus_routes' successfully!")

# Close the database connection
cursor.close()
mydb.close()

# Display the final DataFrame to verify
print("Final DataFrame:")
print(combined_df)

# Establish the connection to MySQL
mydb = mysql.connector.connect(
    host="127.0.0.1",  # Specify the host, commonly localhost
    user="root",       # Your MySQL username
    password="SHYAM@sk25"  # Your MySQL password
)

# Create a cursor object to execute SQL queries
cursor = mydb.cursor()

# Create the database 'REDBUS' if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS REDBUS")
print("Database 'REDBUS' created (or already exists)")

# Use the 'REDBUS' database
cursor.execute("USE REDBUS")

# Define the schema for the MySQL table
schema = """
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT PRIMARY KEY AUTO_INCREMENT,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    price DECIMAL(10,2),
    seats_available INT,
    star_rating FLOAT
);
"""
cursor.execute(schema)
print("Table 'bus_routes' created successfully!")

# List of CSV files to read
csv_files = [
    'himachal_bus_details.csv', 'south_bengal_bus_details.csv', 'uttar_pradesh_bus_details.csv',
    'andhra_pradesh_bus_details.csv', 'WEST_BENGAL_bus_details.csv', 'Telangana_bus_details.csv',
    'rajasthan_bus_details.csv', 'kerala_bus_details.csv', 'chandigarh_bus_details.csv', 'punjab_bus_details.csv'
]

# Combine data from all CSV files into a single DataFrame
dfs = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dfs, ignore_index=True)

# Rename columns to lowercase and match the database schema
combined_df.columns = combined_df.columns.str.lower()  # Normalize column names to lowercase

# Debug: Print column names to verify they match the schema
print("Columns in DataFrame after normalization:", combined_df.columns)

# Rename columns to match the database table schema
combined_df.rename(columns={
    "departing_time": "departing_time",
    "reaching_time": "reaching_time",
    "star_rating": "star_rating",
    "seat_availability": "seats_available",
    "route_name": "route_name",
    "route_link": "route_link",
    "bus_name": "busname",
    "bus_type": "bustype",
    "price": "price",
    "duration": "duration"
}, inplace=True)

# Clean and transform data

# Convert time columns to proper MySQL TIME format
for time_col in ['departing_time', 'reaching_time']:
    if time_col in combined_df.columns:
        combined_df[time_col] = pd.to_datetime(combined_df[time_col], format='%H:%M', errors='coerce').dt.time

# Clean the 'price' column
if 'price' in combined_df.columns:
    combined_df['price'] = (
        combined_df['price'].astype(str)
        .str.replace('INR ', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )

# Extract seat numbers from the 'seats_available' column
if 'seats_available' in combined_df.columns:
    combined_df['seats_available'] = (
        combined_df['seats_available']
        .astype(str)
        .str.extract(r'(\d+)')
        .fillna(0)
        .astype(int)
    )

# Drop rows with missing critical data
critical_columns = ['route_name', 'route_link', 'busname', 'bustype', 'departing_time', 'reaching_time', 'price', 'seats_available', 'star_rating']
combined_df = combined_df.dropna(subset=critical_columns)

# Debug: Check the cleaned DataFrame
print("Cleaned DataFrame preview:")
print(combined_df.head())

# Prepare data for insertion
columns_to_insert = [
    "route_name", "route_link", "busname", "bustype",
    "departing_time", "duration", "reaching_time",
    "price", "seats_available", "star_rating"
]
insert_query = f"""
    INSERT INTO bus_routes ({', '.join(columns_to_insert)}) 
    VALUES ({', '.join(['%s'] * len(columns_to_insert))})
"""
data = combined_df[columns_to_insert].values.tolist()

# Insert data into the 'bus_routes' table
cursor.executemany(insert_query, data)
mydb.commit()
print(f"Data inserted into table 'bus_routes' successfully!")

# Close the database connection
cursor.close()
mydb.close()

# Display the final DataFrame to verify
print("Final DataFrame:")
print(combined_df)
